<a href="https://colab.research.google.com/github/Venkata1345/NLP_LLM_implementations/blob/main/RoPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
def apply_rope(embeddings,base=10000.0):
  seq_len,dim=embeddings.shape
  positions=np.arange(seq_len,dtype=float).reshape(seq_len,1)
  half_dim=dim//2
  i_values=np.arange(half_dim,dtype=float)
  inv_freq = base ** (- (2 * i_values) / dim) # Shape: (dim/2,)
  inv_freq = inv_freq.reshape(1, half_dim)    # Shape: (1, dim/2)
  angles=positions*inv_freq
  cos_angles=np.cos(angles)
  sin_angles=np.sin(angles)
  x_even=embeddings[:,0::2]
  x_odd=embeddings[:,1::2]
  rotated_x_even=x_even*cos_angles-x_odd*sin_angles
  rotated_x_odd=x_odd*cos_angles+x_even*sin_angles
  rotated_embeddings = np.empty_like(embeddings)
  rotated_embeddings[:, 0::2] = rotated_x_even
  rotated_embeddings[:, 1::2] = rotated_x_odd
  return rotated_embeddings

if __name__ == "__main__":
    # Configuration
    seq_len = 5
    dim = 8  # Must be even
    rope_base = 10000.0

    print(f"Configuration: seq_len={seq_len}, dim={dim}, base={rope_base}\n")
    q_original = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=float)
    k_original = np.array([0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1], dtype=float)
    print("Original q vector:", q_original)
    print("Original k vector:", k_original)
    print("-" * 30)

    example_embeddings = np.zeros((seq_len, dim))
    example_embeddings[0, :] = q_original # q at pos 0
    example_embeddings[1, :] = k_original # k at pos 1
    example_embeddings[2, :] = q_original # q at pos 2
    example_embeddings[3, :] = k_original # k at pos 3
    example_embeddings[4, :] = q_original # q at pos 4

    rotated_example_embeddings=apply_rope(example_embeddings,base=rope_base)
        # Extract the rotated vectors
    q_pos0_rope = rotated_example_embeddings[0, :]
    k_pos1_rope = rotated_example_embeddings[1, :]
    q_pos2_rope = rotated_example_embeddings[2, :]
    k_pos3_rope = rotated_example_embeddings[3, :]
    q_pos4_rope = rotated_example_embeddings[4, :]

    print("\n--- Rotated Vectors (examples) ---")
    print(f"q at pos 0 (RoPE applied): {q_pos0_rope[:4]}...") # Print first 4 elements for brevity
    print(f"k at pos 1 (RoPE applied): {k_pos1_rope[:4]}...")
    print(f"q at pos 2 (RoPE applied): {q_pos2_rope[:4]}...")
    print(f"k at pos 3 (RoPE applied): {k_pos3_rope[:4]}...")
    print("-" * 30)



Configuration: seq_len=5, dim=8, base=10000.0

Original q vector: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8]
Original k vector: [0.8 0.7 0.6 0.5 0.4 0.3 0.2 0.1]
------------------------------

--- Rotated Vectors (examples) ---
q at pos 0 (RoPE applied): [0.1 0.2 0.3 0.4]...
k at pos 1 (RoPE applied): [-0.15678784  1.0513884   0.54708579  0.55740213]...
q at pos 2 (RoPE applied): [-0.22347417  0.00770038  0.21455224  0.45162743]...
k at pos 3 (RoPE applied): [-0.890778   -0.58009874  0.42544179  0.65498037]...
------------------------------
